### Import the required libraries and modules:

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
import numpy as np

In [ ]:
from tqdm import tqdm
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
%cd /content/gdrive/MyDrive/medical_image_processing/

Mounted at /content/gdrive
/content/gdrive/MyDrive/medical_image_processing


In [ ]:
from fundal_image.utils import create_dir, get_filenames_sorted
from patches import generate_patches_images_only, get_zero_padding_size, reconstruct_patches
from metrics import iou, dice_coef, dice_loss, DiceBCELoss, bce
from metrics import compute_metrics, save_results
from utils import read_image_train, read_mask_train

In [ ]:
image_sizes = {"HRF": (2336, 3504), "drive": (584, 565), "chase_db": (960, 999)}
patch_size = {"HRF": (512, 512)}

dataset_name = "HRF"
patch_size = patch_size[dataset_name]
patch_height = patch_size[0]
patch_width = patch_size[1]

In [ ]:
loss_str = "dice_loss"
with_augmentation = False
stride=512
seed = 42
run_id = 1
test_percentage = 0.4
class_weight = 0.6
class_weight_str = str(class_weight).replace(".", "_")
aug_string = ""
if with_augmentation:
    aug_string = "_hvr"
test_percentage_str = str(test_percentage).replace(".", "_")
root_path = f"/content/datasets/hrf_{patch_height}_{stride}_{loss_str}_{class_weight_str}_{aug_string}_{test_percentage_str}_{run_id}/"
images_path = f"{root_path}/images/"
gt_path = f"{root_path}/groundtruth"

create_dir(images_path)
create_dir(gt_path)

In [ ]:
def wbce(actual, predicted):
    y_pred = K.clip(predicted, K.epsilon(), 1 - K.epsilon())
    term_0 = (1 - actual) * K.log(1 - y_pred)
    term_1 = class_weight * actual * K.log(y_pred + K.epsilon())
    return -K.mean(term_0 + term_1, axis=0)

In [ ]:
root_path

'/content/datasets/hrf_512_512_dice_loss_0_6__0_4_1/'

In [ ]:
!cp /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets/HRF/images/*.* {images_path}
!cp /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets/HRF/groundtruth/*.* {gt_path}



In [ ]:
images_files, gt_files = get_filenames_sorted(images_path, gt_path)
print(len(images_files), len(gt_files))
# train_test_split
from sklearn.model_selection import train_test_split
label = [_xtrain.rsplit("/", 1)[1].rsplit(".", 1)[0].rsplit("_", 1)[1] for _xtrain in images_files] 
xtrain, xtest, ytrain, ytest = train_test_split(images_files, gt_files, stratify=label,  test_size=test_percentage, random_state=seed)

45 45


In [ ]:
print(len(xtrain), len(xtest))
print(label)

27 18
['dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h']


# Generate Patches

In [ ]:
num_pixels_to_add = [0, 0]
if (image_sizes[dataset_name][0] % patch_size[0]) > 0:
    num_pixels_to_add[0] = ((image_sizes[dataset_name][0] // patch_size[0]) + 1) * patch_size[0]
    num_pixels_to_add[1] = ((image_sizes[dataset_name][1] // patch_size[1]) + 1) * patch_size[1]

split_name = "split_2"
data_path = root_path
splits =["train","test"]


split = "train"
#for split in splits:
image_patches_path = f"{data_path}/{split_name}/{split}/image/"
create_dir(image_patches_path)
gt_patches_path = f"{data_path}/{split_name}/{split}/mask/"
create_dir(gt_patches_path)
im_patches_dict_train, metadata_train_df, gt_patches_dict_train = generate_patches_images_only(xtrain,
                                                                image_patches_path,
                                                                patch_size=patch_size,
                                                                stride=stride,
                                                                zero_padding_size=num_pixels_to_add,
                                                                gt_files=ytrain,
                                                                save_path_gt=gt_patches_path)
metadata_train_df.to_csv(f"{data_path}/{split_name}/image_patches_{stride}_{stride}_{split}.csv", index=False)



Generating patches for 27 images


100%|██████████| 27/27 [00:15<00:00,  1.70it/s]


In [ ]:
num_zero_padding_height, num_zero_padding_width = get_zero_padding_size(patch_size,
                                                                        image_sizes[dataset_name])
height_odd = 0 if (num_zero_padding_height % 2) == 0 else 1
width_odd = 0 if (num_zero_padding_width % 2) == 0 else 1


In [ ]:
print(num_zero_padding_height,num_zero_padding_width, height_odd, width_odd)

224 80 0 0


In [ ]:
split = "test"
#for split in splits:
image_patches_path = f"{data_path}/{split_name}/{split}/image/"
create_dir(image_patches_path)
gt_patches_path = f"{data_path}/{split_name}/{split}/mask/"
create_dir(gt_patches_path)
im_patches_dict_test, metadata_test_df, gt_patches_dict_test = generate_patches_images_only(xtest,
                                                                image_patches_path,
                                                                patch_size=patch_size,
                                                                stride=stride,
                                                                zero_padding_size=num_pixels_to_add,
                                                                gt_files=ytest,
                                                                save_path_gt=gt_patches_path)
metadata_test_df.to_csv(f"{data_path}/{split_name}/image_patches_{patch_height}_{stride}_{split}.csv", index=False)


Generating patches for 18 images


100%|██████████| 18/18 [00:09<00:00,  1.80it/s]


In [ ]:
print(metadata_test_df.shape)
metadata_train_df.head()

(630, 7)


,image_filename,row,col,patch_height,patch_width,stride,num_fg_pixels
0,/content/datasets/hrf_512_512_dice_loss_0_6__0...,0,0,512,512,512,103562
1,/content/datasets/hrf_512_512_dice_loss_0_6__0...,0,512,512,512,512,199885
2,/content/datasets/hrf_512_512_dice_loss_0_6__0...,0,1024,512,512,512,204800
3,/content/datasets/hrf_512_512_dice_loss_0_6__0...,0,1536,512,512,512,204800
4,/content/datasets/hrf_512_512_dice_loss_0_6__0...,0,2048,512,512,512,204800


# Reconstruct test images and ground truths

In [ ]:
import glob
split = "test"
image_patches_path = f"{data_path}/{split_name}/{split}/image/"

reconstructed_original_path = f"{data_path}/{split_name}/reconstructed/{split}/image"
create_dir(reconstructed_original_path)
reconstruct_patches(im_patches_dict_test,
                    metadata_test_df,
                    reconstructed_original_path
                    )


In [ ]:
reconstructed_gt_path = f"{data_path}/{split_name}/reconstructed/{split}/mask"

create_dir(reconstructed_gt_path)
reconstruct_patches(gt_patches_dict_test,
                    metadata_test_df,
                    reconstructed_gt_path,
                    )


In [ ]:
print([k for k in im_patches_dict_test.keys()])

['11_dr', '05_dr', '07_dr', '01_h', '15_g', '06_h', '15_h', '04_dr', '10_h', '06_g', '02_g', '05_g', '13_dr', '09_h', '03_g', '08_dr', '08_g', '12_h']


# Construct the Model:

In [ ]:
from fundal_image.blood_vessel_segmentation.model import build_unet

In [ ]:

input_shape = (512, 512, 3)
model = build_unet(input_shape)
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                              

# Training the Model:

In [ ]:
if loss_str == "bce":
    loss = bce
elif loss_str == "dice_bce":
    loss = DiceBCELoss
elif loss_str == "dice_loss":
    loss = dice_loss
elif loss_str == "wbce":
    loss = wbce
else:
    raise Exception("Invalid value for loss_str")
    
print(loss)

<function dice_loss at 0x7f67e59c35f0>


In [ ]:
H = patch_height
W = patch_width

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.png")))
    y = sorted(glob(os.path.join(path, "mask", "*.png")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

In [ ]:
from keras.metrics import AUC

In [ ]:
model_dir = f"{root_path}/trained_models/"
create_dir(model_dir)

In [ ]:
!ls {model_dir}

In [ ]:
%cd /content/datasets

/content/datasets


In [ ]:
from glob import glob

In [ ]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory to save files """
create_dir(model_dir)

""" Hyperparameters """
batch_size = 4
lr = 1e-4
num_epochs = 50
model_path = os.path.join(f"{model_dir}", "model.h5")
csv_path = os.path.join(f"{model_dir}", "data.csv")

""" Dataset """
dataset_path = f"{data_path}/{split_name}"
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "test")

train_x, train_y = load_data(train_path)
train_x, train_y = shuffling(train_x, train_y)
valid_x, valid_y = load_data(valid_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

train_steps = len(train_x)//batch_size
valid_setps = len(valid_x)//batch_size

if len(train_x) % batch_size != 0:
    train_steps += 1
if len(valid_x) % batch_size != 0:
    valid_setps += 1

""" Model """
model = build_unet((H, W, 3))
model.compile(loss=loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision(), AUC()])

callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
]

model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_setps,
    callbacks=callbacks
)

Train: 945 - 945
Valid: 630 - 630
Epoch 1/50
237/237 [==============================] - ETA: 0s - loss: 0.5472 - dice_coef: 0.4521 - iou: 0.3037 - recall: 0.7950 - precision: 0.3989 - auc: 0.9175
Epoch 1: val_loss improved from inf to 0.92677, saving model to /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//trained_models/model.h5
237/237 [==============================] - 103s 357ms/step - loss: 0.5472 - dice_coef: 0.4521 - iou: 0.3037 - recall: 0.7950 - precision: 0.3989 - auc: 0.9175 - val_loss: 0.9268 - val_dice_coef: 0.0733 - val_iou: 0.0381 - val_recall: 4.4193e-07 - val_precision: 0.0011 - val_auc: 0.5741 - lr: 1.0000e-04
Epoch 2/50
237/237 [==============================] - ETA: 0s - loss: 0.3761 - dice_coef: 0.6230 - iou: 0.4587 - recall: 0.8058 - precision: 0.7055 - auc: 0.9420
Epoch 2: val_loss improved from 0.92677 to 0.63194, saving model to /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//trained_models/model.h5
237/237 [==============================] - 81s 343ms/s

# Evaluating the Model:

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from tensorflow.keras.models import load_model

In [ ]:

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(int)
    return ori_x, x

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.png")))
    y = sorted(glob(os.path.join(path, "mask", "*.png")))
    return x, y

def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

In [ ]:
import glob
supported_extensions = ["png", "jpg", "tif", "jpeg", "gif", "bmp"]

def image_files(path, file_prefix=None):
    all_image_files = []
    # print("path", path)
    # print("file_prefix", file_prefix)
    if file_prefix is None:
        file_names = list(glob.glob(path + "/*.*" ))
    else:
        prefix___ = path + "/" + file_prefix + ".*"
        #print("prefix", prefix___)
        file_names = list(glob.glob(prefix___))

    # print("filenames", file_names)

    for filename in file_names:
        ext = filename.rsplit(".", 1)[1]
        if ext.lower() in supported_extensions:
            all_image_files.append(filename)
    # print("All Image files", all_image_files)
    return sorted(all_image_files)


In [ ]:
results_path = f"{root_path}/results/"
prediction_path = f"{root_path}/predictions/"
prediction_np_path = f"{root_path}/predictions_np/"
create_dir(prediction_path)
create_dir(prediction_np_path)
create_dir(results_path)

In [ ]:
results_path

'/content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//results/'

In [ ]:
from glob import glob

In [ ]:
from copy import deepcopy
from sklearn.metrics import roc_auc_score
# prediction_path = "predictions"

""" Save the results in this folder """
# create_dir("results_sunil_test")
# create_dir(prediction_path)

""" Load the model """
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
model_path = os.path.join(f"{model_dir}", "model.h5")

model = load_model(model_path, compile=False)

""" Load the dataset """
dataset_path = os.path.join(f"{data_path}/{split_name}", "test")
test_x, test_y = load_data(dataset_path)

""" Make the prediction and calculate the metrics values """
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.split("/")[-1].split(".")[0]

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)
    y = y.astype(np.int)

    """ Prediction """
    y_pred = model.predict(np.expand_dims(x, axis=0))[0]
    np.save(f"{prediction_np_path}/{name}.npy", y_pred)
    y_prob = deepcopy(y_pred).flatten()
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(int)
    y_pred = np.squeeze(y_pred, axis=-1)

    """ Saving the images """
    save_image_path = f"{results_path}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path)
    cv2.imwrite(f"{prediction_path}/{name}.png", y_pred * 255)
    

    """ Flatten the array """
    y = y.flatten()
    y_pred = y_pred.flatten()

    """ Calculate the metrics """
    acc_value = accuracy_score(y, y_pred)
    f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
    precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
    if np.sum(y) == 0 or np.sum(y) == H * W:
        auc_value = 1
    else:
        auc_value = roc_auc_score(y, y_prob)
    dice_value = dice_coef(y.astype(np.float64), y_prob)

    SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, auc_value, dice_value])

score = [s[1:] for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.5f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")
print(f"AUC: {score[5]:0.5f}")
print(f"Dice Coeff: {score[6]:0.5f}")
""" Saving """
df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "AUC", "Dice Coeff"])
df.to_csv(f"{results_path}/score.csv")

  0%|          | 0/630 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  0%|          | 1/630 [00:00<09:44,  1.08it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to rev

Accuracy: 0.97483
F1: 0.74666
Jaccard: 0.62028
Recall: 0.73066
Precision: 0.79305
AUC: 0.89781
Dice Coeff: 0.74451


# Reconstruct original image

In [ ]:
prediction_np_path

'/content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//predictions_np/'

In [ ]:
from glob import glob

In [ ]:
# Create patches dictionary
predicted_patches_dict = dict()
for file in glob(f"{prediction_np_path}/*.npy"):
    print("file", file)
    patch = np.load(file)
    fname_without_ext = file.rsplit("/", 1)[1].rsplit(".", 1)[0]
    fname_without_ext = fname_without_ext[1:]
    patch_number = int(fname_without_ext.rsplit("_", 1)[1])
    image_file = fname_without_ext.rsplit("_", 1)[0]
    print(image_file, patch_number )

    if image_file in predicted_patches_dict.keys():
        predicted_patches_dict[image_file][patch_number] = np.load(file)
    else:
        patches = [None] * 140
        patches[patch_number] = np.load(file)
        predicted_patches_dict[image_file] = patches

file /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//predictions_np/_07_dr_8.npy
07_dr 8
file /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//predictions_np/_04_dr_30.npy
04_dr 30
file /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//predictions_np/_04_dr_28.npy
04_dr 28
file /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//predictions_np/_06_h_14.npy
06_h 14
file /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//predictions_np/_10_h_20.npy
10_h 20
file /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//predictions_np/_01_h_27.npy
01_h 27
file /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//predictions_np/_08_dr_24.npy
08_dr 24
file /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//predictions_np/_08_dr_34.npy
08_dr 34
file /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//predictions_np/_08_dr_17.npy
08_dr 17
file /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//predictions_np/_13_dr_24.npy
13_dr 24
file /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//p

In [ ]:
reconstructed_predictions_path = f"{data_path}/{split_name}/reconstructed/{split}/predictions_reconstructed"
create_dir(reconstructed_predictions_path)
reconstruct_patches(predicted_patches_dict,
                    metadata_test_df,
                    reconstructed_predictions_path,
                    mfactor=255
                    )


In [ ]:
!pwd

/content/datasets


In [ ]:
def read_mask(path):
    print(path)
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
#    x = x.astype(np.int32)
    return ori_x, x

In [ ]:
def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((ori_x.shape[0], 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    print(f"Saving image of size {cat_images.shape} to {save_image_path}")
    cv2.imwrite(save_image_path, cat_images)

# Remove the zero padding 

In [ ]:
results_path_reconstructed_orig_size = f"{root_path}/results_reconstructed_orig_size/"
gt_path_reconstructed_orig_size = f"{root_path}/reconstructed_orig_size/mask/"
im_path_reconstructed_orig_size = f"{root_path}/reconstructed_orig_size/image/"

create_dir(results_path_reconstructed_orig_size)
create_dir(gt_path_reconstructed_orig_size)
create_dir(im_path_reconstructed_orig_size)


In [ ]:
reconstructed_pred_files, _ = get_filenames_sorted(reconstructed_predictions_path)
print(len(reconstructed_pred_files))
print(reconstructed_pred_files[:5])
for reconstructed_pred_file in reconstructed_pred_files:
    im = cv2.imread(reconstructed_pred_file)
    name = reconstructed_pred_file.rsplit("/", 1)[1]
    im_cropped = im[num_zero_padding_height // 2 : - num_zero_padding_height // 2, num_zero_padding_width // 2 : -num_zero_padding_width //2 ]
    print(im.shape, im_cropped.shape)
    cv2.imwrite(f"{results_path_reconstructed_orig_size}/{name}", im_cropped)

18
['/content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//split_2/reconstructed/test/predictions_reconstructed/01_h.png', '/content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//split_2/reconstructed/test/predictions_reconstructed/02_g.png', '/content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//split_2/reconstructed/test/predictions_reconstructed/03_g.png', '/content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//split_2/reconstructed/test/predictions_reconstructed/04_dr.png', '/content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//split_2/reconstructed/test/predictions_reconstructed/05_dr.png']
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 35

In [ ]:
gt_path_reconstructed = f"{root_path}/{split_name}/reconstructed/{split}/"

reconstructed_im_files, reconstructed_gt_files = get_filenames_sorted(f"{gt_path_reconstructed}/image/", f"{gt_path_reconstructed}/mask/" )
print(len(reconstructed_gt_files))
print(reconstructed_gt_files[:5])

for reconstructed_im_file, reconstructed_gt_file in zip(reconstructed_im_files, reconstructed_gt_files):
    gt = cv2.imread(reconstructed_gt_file)
    im = cv2.imread(reconstructed_im_file)
    name = reconstructed_gt_file.rsplit("/", 1)[1]
    gt_cropped = gt[num_zero_padding_height // 2 : - num_zero_padding_height // 2, num_zero_padding_width // 2 : -num_zero_padding_width //2 ]
    im_cropped = im[num_zero_padding_height // 2 : - num_zero_padding_height // 2, num_zero_padding_width // 2 : -num_zero_padding_width //2 ]
    print(im.shape, im_cropped.shape)
    print(gt.shape, gt_cropped.shape)

    cv2.imwrite(f"{gt_path_reconstructed_orig_size}/{name}", gt_cropped)
    cv2.imwrite(f"{im_path_reconstructed_orig_size}/{name}", im_cropped)


18
['/content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//split_2/reconstructed/test//mask/01_h.png', '/content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//split_2/reconstructed/test//mask/02_g.png', '/content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//split_2/reconstructed/test//mask/03_g.png', '/content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//split_2/reconstructed/test//mask/04_dr.png', '/content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//split_2/reconstructed/test//mask/05_dr.png']
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 3)
(2560, 3584, 3) (2336, 3504, 

In [ ]:
!ls {gt_path_reconstructed}

image  mask  predictions_reconstructed


In [ ]:
gt_path_reconstructed

'/content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//split_2/reconstructed/test/'

In [ ]:
!ls {results_path_reconstructed_orig_size}

01_h.png  04_dr.png  06_g.png	08_dr.png  10_h.png   13_dr.png
02_g.png  05_dr.png  06_h.png	08_g.png   11_dr.png  15_g.png
03_g.png  05_g.png   07_dr.png	09_h.png   12_h.png   15_h.png


In [ ]:
results_path_reconstructed = f"{root_path}/results_reconstructed/"
create_dir(results_path_reconstructed)

In [ ]:
from glob import glob

In [ ]:
!ls {dataset_path}/mask

_01_h_0.png    _04_dr_29.png  _06_h_16.png   _08_g_3.png    _12_h_22.png
_01_h_10.png   _04_dr_2.png   _06_h_17.png   _08_g_4.png    _12_h_23.png
_01_h_11.png   _04_dr_30.png  _06_h_18.png   _08_g_5.png    _12_h_24.png
_01_h_12.png   _04_dr_31.png  _06_h_19.png   _08_g_6.png    _12_h_25.png
_01_h_13.png   _04_dr_32.png  _06_h_1.png    _08_g_7.png    _12_h_26.png
_01_h_14.png   _04_dr_33.png  _06_h_20.png   _08_g_8.png    _12_h_27.png
_01_h_15.png   _04_dr_34.png  _06_h_21.png   _08_g_9.png    _12_h_28.png
_01_h_16.png   _04_dr_3.png   _06_h_22.png   _09_h_0.png    _12_h_29.png
_01_h_17.png   _04_dr_4.png   _06_h_23.png   _09_h_10.png   _12_h_2.png
_01_h_18.png   _04_dr_5.png   _06_h_24.png   _09_h_11.png   _12_h_30.png
_01_h_19.png   _04_dr_6.png   _06_h_25.png   _09_h_12.png   _12_h_31.png
_01_h_1.png    _04_dr_7.png   _06_h_26.png   _09_h_13.png   _12_h_32.png
_01_h_20.png   _04_dr_8.png   _06_h_27.png   _09_h_14.png   _12_h_33.png
_01_h_21.png   _04_dr_9.png   _06_h_28.png   _09_h_1

In [ ]:
!ls {results_path_reconstructed}

In [ ]:
!ls {reconstructed_predictions_path}

01_h.png  04_dr.png  06_g.png	08_dr.png  10_h.png   13_dr.png
02_g.png  05_dr.png  06_h.png	08_g.png   11_dr.png  15_g.png
03_g.png  05_g.png   07_dr.png	09_h.png   12_h.png   15_h.png


In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, recall_score, precision_score, roc_auc_score, confusion_matrix

from utils import read_image, read_mask
from utils import get_filenames_sorted
from tqdm import tqdm
from copy import deepcopy
smooth = 1e-15

def compute_metrics(data_path: str,
                    results_path: str,
                    threshold: float,
                    model=None,
                    predictions_path: str = None,
                    prediction_np_path=None):
    """ Make the prediction and calculate the metrics values
    data_path: Path location where images and groundtruths are saved. Images should be saved in a folder data_path/image
     and ground truths should be saved at data_path/mask
    result_path: Path to directory where results should be saved

    """

    """ Load the dataset """
    test_x, test_y = get_filenames_sorted(f"{data_path}/image", f"{data_path}/mask")
    SCORE = []
    print("Threshold", threshold)
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        if model is None and predictions_path is None:
            raise Exception("Invalid input parameters. Both model and prediction_path can not be None")
        """ Extracting name """
        name = x.split("/")[-1].split(".")[0]

        """ Read the image and mask """
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)
        y = y.astype(np.int)
#        print(name)

        #     """ Prediction """
        if model != None:
            y_pred = model.predict(np.expand_dims(x, axis=0))[0]
        else:
#            print("Reading predicted image from ", f"{predictions_path}/{name}.png")
            orig_y_pred, y_pred = read_mask(f"{predictions_path}/{name}.png")
#            print("orig_y_pred",orig_y_pred.shape, np.max(orig_y_pred), np.min(orig_y_pred))
#            print("ypred", y_pred.shape, np.max(y_pred), np.min(y_pred))

        if prediction_np_path is not None:
            np.save(f"{prediction_np_path}/{name}.npy", y_pred)

        y_prob = deepcopy(y_pred).flatten()
        y_pred = y_pred > threshold
        y_pred = y_pred.astype(int)
        y_pred = np.squeeze(y_pred)

        """ Saving the images """
        save_image_path = f"{results_path}/{name}.png"
 #       print(ori_x.shape, ori_y.shape, y_pred.shape)
#        print("ori_y",ori_y.shape, np.min(ori_y), np.max(ori_y))
#        print("y", y.shape, np.min(y), np.max(y))
        save_results(ori_x, ori_y, y_pred, save_image_path)

        if model is not None:
            cv2.imwrite(f"{predictions_path}/{name}.png", y_pred * 255)

        """ Flatten the array """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculate the metrics """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
        confusion_metrics = confusion_matrix(y, y_pred, labels=[0, 1])
        if np.sum(y) == 0 or np.sum(y) == y.shape[0] :
            auc_value = 1
        else:
            auc_value = roc_auc_score(y, y_prob)
        dice_value = dice_coef(y.astype(np.float64), y_prob.astype(np.float64))
        SCORE.append([name, threshold, acc_value, f1_value, jac_value, recall_value, precision_value, auc_value, dice_value, confusion_metrics[0,0], confusion_metrics[0,1],confusion_metrics[1,0],confusion_metrics[1,1]])

    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Threshold: {threshold:0.5f}")
    print(f"Accuracy: {score[1]:0.5f}")
    print(f"F1: {score[2]:0.5f}")
    print(f"Jaccard: {score[3]:0.5f}")
    print(f"Recall: {score[4]:0.5f}")
    print(f"Precision: {score[5]:0.5f}")
    print(f"AUC: {score[6]:0.5f}")
    print(f"Dice Coeff: {score[7]:0.5f}")

    """ Saving """
    df = pd.DataFrame(SCORE, columns=["Image","Threshold", "Acc", "F1", "Jaccard", "Recall", "Precision", "AUC", "Dice Coef", "TN", "FP","FN","TP"])
    threshold_str = str(threshold).replace(".", "_")
    df.to_csv(f"{results_path}/score_full_{threshold_str}.csv")
    return df, score

In [ ]:
dataset_path = f"{root_path}/reconstructed_orig_size/"
df = dict()
score = dict()
for threshold in range(4,7):
    df[threshold], score[threshold] = compute_metrics(dataset_path,
                        results_path=results_path_reconstructed,
                        threshold = threshold / 10,
                        model=None,
                        predictions_path=results_path_reconstructed_orig_size,
                        prediction_np_path=None)

Threshold 0.4


  0%|          | 0/18 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Saving image of size (2336, 10532, 3) to /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//results_reconstructed//01_h.png


  6%|▌         | 1/18 [00:19<05:26, 19.23s/it]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Saving image of size (2336, 10532, 3) to /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//results_reconstructed//02_g.png


...

Saving image of size (2336, 10532, 3) to /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//results_reconstructed//15_g.png


 94%|█████████▍| 17/18 [05:18<00:18, 18.59s/it]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Saving image of size (2336, 10532, 3) to /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//results_reconstructed//15_h.png


100%|██████████| 18/18 [05:37<00:00, 18.74s/it]

Threshold: 0.60000
Accuracy: 0.97166
F1: 0.80022
Jaccard: 0.66802
Recall: 0.75623
Precision: 0.85767
AUC: 0.87291
Dice Coeff: 0.80022


In [ ]:
dataset_path = f"{root_path}/{split_name}/reconstructed/{split}/"
df = dict()
score = dict()
for threshold in range(4,7):
    df[threshold], score[threshold] = compute_metrics(dataset_path,
                        results_path=results_path_reconstructed,
                        threshold = threshold / 10,
                        model=None,
                        predictions_path=reconstructed_predictions_path,
                        prediction_np_path=None)

Threshold 0.4


  0%|          | 0/18 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//results_reconstructed//01_h.png


  6%|▌         | 1/18 [00:20<05:54, 20.86s/it]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//results_reconstructed//02_g.png


...

Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//results_reconstructed//15_g.png


 94%|█████████▍| 17/18 [05:59<00:20, 20.69s/it]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//results_reconstructed//15_h.png


100%|██████████| 18/18 [06:20<00:00, 21.12s/it]

Threshold: 0.60000
Accuracy: 0.97472
F1: 0.80022
Jaccard: 0.66802
Recall: 0.75623
Precision: 0.85767
AUC: 0.87351
Dice Coeff: 0.80022


In [ ]:
results_path_reconstructed

'/content/datasets/hrf_512_512_dice_loss_0_6__0_4_1//results_reconstructed/'

In [ ]:
!ls 

hrf_512_512_dice_loss_0_6__0_4_1  logs


In [ ]:
folder_name = root_path.split("/")[3]
print(folder_name)

hrf_512_512_dice_loss_0_6__0_4_1


In [ ]:
!zip -r {folder_name}.zip  {folder_name}/

Streaming output truncated to the last 5000 lines.
  adding: hrf_512_512_dice_loss_0_6__0_4_1/results/_05_dr_32.png (deflated 8%)
  adding: hrf_512_512_dice_loss_0_6__0_4_1/results/_11_dr_17.png (deflated 7%)
  adding: hrf_512_512_dice_loss_0_6__0_4_1/results/_11_dr_12.png (deflated 5%)
  adding: hrf_512_512_dice_loss_0_6__0_4_1/results/_03_g_23.png (deflated 7%)
  adding: hrf_512_512_dice_loss_0_6__0_4_1/results/_05_g_19.png (deflated 5%)
  adding: hrf_512_512_dice_loss_0_6__0_4_1/results/_15_h_14.png (deflated 7%)
  adding: hrf_512_512_dice_loss_0_6__0_4_1/results/_07_dr_16.png (deflated 6%)
  adding: hrf_512_512_dice_loss_0_6__0_4_1/results/_06_g_2.png (deflated 8%)
  adding: hrf_512_512_dice_loss_0_6__0_4_1/results/_11_dr_28.png (deflated 14%)
  adding: hrf_512_512_dice_loss_0_6__0_4_1/results/_02_g_25.png (deflated 7%)
  adding: hrf_512_512_dice_loss_0_6__0_4_1/results/_08_dr_12.png (deflated 4%)
  adding: hrf_512_512_dice_loss_0_6__0_4_1/results/_08_g_0.png (deflated 13%)
  addin

In [ ]:
!ls -ltrh


total 1.7G
drwxr-xr-x  4 root root 4.0K Mar 15 08:13 logs
drwxr-xr-x 12 root root 4.0K Mar 15 09:01 hrf_512_512_dice_loss_0_6__0_4_1
-rw-r--r--  1 root root 1.7G Mar 15 09:39 hrf_512_512_dice_loss_0_6__0_4_1.zip


In [ ]:
!cp {folder_name}.zip /content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches_with_fixes/

In [ ]:
%cd /content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches_with_fixes/

/content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches_with_fixes


In [ ]:
!pwd

/content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches_with_fixes


In [ ]:
!ls

hrf_512_512_dice_bce_0_6__0_4_1       hrf_512_512_wbce_0_6__0_4_1.zip
hrf_512_512_dice_bce_0_6__0_4_1.zip   hrf_768_768_wbce_0_6__0_4_1
hrf_512_512_dice_loss_0_6__0_4_1.zip  hrf_768_768_wbce_0_6__0_4_1.zip
hrf_512_512_wbce_0_6__0_4_1


In [ ]:
!unzip {folder_name}.zip


Streaming output truncated to the last 5000 lines.
  inflating: hrf_512_512_dice_loss_0_6__0_4_1/results/_05_dr_32.png  
  inflating: hrf_512_512_dice_loss_0_6__0_4_1/results/_11_dr_17.png  
  inflating: hrf_512_512_dice_loss_0_6__0_4_1/results/_11_dr_12.png  
  inflating: hrf_512_512_dice_loss_0_6__0_4_1/results/_03_g_23.png  
  inflating: hrf_512_512_dice_loss_0_6__0_4_1/results/_05_g_19.png  
  inflating: hrf_512_512_dice_loss_0_6__0_4_1/results/_15_h_14.png  
  inflating: hrf_512_512_dice_loss_0_6__0_4_1/results/_07_dr_16.png  
  inflating: hrf_512_512_dice_loss_0_6__0_4_1/results/_06_g_2.png  
  inflating: hrf_512_512_dice_loss_0_6__0_4_1/results/_11_dr_28.png  
  inflating: hrf_512_512_dice_loss_0_6__0_4_1/results/_02_g_25.png  
  inflating: hrf_512_512_dice_loss_0_6__0_4_1/results/_08_dr_12.png  
  inflating: hrf_512_512_dice_loss_0_6__0_4_1/results/_08_g_0.png  
  inflating: hrf_512_512_dice_loss_0_6__0_4_1/results/_08_g_9.png  
  inflating: hrf_512_512_dice_loss_0_6__0_4_1/res

In [ ]:
drive.flush_and_unmount()